# Interpolation datapath generator

### Parameters

In [9]:
import math 
from datetime import datetime

In [10]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS
BUFFER_LINES = 9                                # Number of lines in the transposed buffer
BUFFER_COLUMNS = 4                              # Number of columns in the transposed buffer


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)


####################################################
# INTERPOLATOR DATAPATH DEFINITIONS
####################################################
PARALLEL = 1                                    # Number of lines to be processed each time the interpolator runs. 
                                                #       Obs: 1 line has 9 integer samples
SAMPLES_PER_LINE = 4                            # Number of samples in one line
INDEX_0 = 2                                     # Which position to be considered to input the buffer
INT_FILE_NAME = 'interpolation_datapath'        # Interpolation datapath file name
INT_DESCRIPTION = 'Interpolation datapath to process '


### Intermediate variables calculation

In [ ]:
####################################################
# INTERPOLATOR VARIABLES
####################################################
# Modified date
DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

# Inputs from control and MV Gen datapath [signal name, bit width, io type, description]
# Obs: io type is relative to this datapath pov
INT_CTRL_SIGNALS = [
    ['RST_ASYNC_INTERP', 1, 'input', '\t\t\t//Async reset for regs INPUT_LINE, INT_OUT and BUFFER.'],
    ['RST_ASYNC_REG_COUNTER', 1, 'input', ''],

    ['WRITE_REG_INPUT_LINE', 1, 'input', ''],
    ['WRITE_REG_COUNTER', 1, 'input', ''],
    ['WRITE_REG_INT_OUT', 1, 'input', ''],
    ['WRITE_BUFFER', 1, 'input', ''],

    ['SEL_BUFFER_IN', 1, 'input', '\t\t\t/0: 1st pass output; 1: OUT_REG_INPUT_LINE'],
    ['SEL_INTERP_IN', 1, 'input', '\t\t\t/0: buffer; 1: OUT_REG_INPUT_LINE'],
    ['SEL_DIMENSION', 1, 'input', '\t\t\t/0: Y; 1: X'],

    ['INTEGER_SAMPLES', PIXEL_WIDTH*INPUT_LINE_SIZE, 'input unsigned', ''],
    ['FRAC_MV_X',  int(math.log2((FILTERS+1))), 'input unsigned', '\t\t\t// Fractional part of MV_X from MV Generator.'],
    ['FRAC_MV_Y',  int(math.log2((FILTERS+1))), 'input unsigned', '\t\t\t// Fractional part of MV_Y from MV Generator.'],

    ['LOOP_3', 1, 'wire output', ''],
    ['LOOP_4', 1, 'wire output', ''],
    ['LOOP_9', 1, 'wire output', ''],
    ['LOOP_13', 1, 'wire output', ''],

    ['INTERP_OUT_0', OUT_2ND_INTERP_WIDTH, 'wire output', ''],
    ['INTERP_OUT_1', OUT_2ND_INTERP_WIDTH, 'wire output', ''],
    ['INTERP_OUT_2', OUT_2ND_INTERP_WIDTH, 'wire output', ''],
    ['INTERP_OUT_3', OUT_2ND_INTERP_WIDTH, 'wire output', '']
]


# Intermediary signals 
# Template: [signal name, signal type, signal width, description]
# Obs: signal type must be in accordance with the component's signal type
INT_SIGNALS = [
    ['OUT_REG_INPUT_LINE', 'wire unsigned', PIXEL_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_REG_COUNTER', 'wire unsigned', 4, ''],
    ['OUT_BUFFER_0', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_1', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_2', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_BUFFER_3', 'wire signed', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],

    ['OUT_MUX_SEL_BUF_IN_0', 'wire unsigned', OUT_1ST_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_1', 'wire unsigned', OUT_1ST_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_2', 'wire unsigned', OUT_1ST_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUF_IN_3', 'wire unsigned', OUT_1ST_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_BUFFER_COLUMN', 'wire unsigned', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_MUX_SEL_INTERP_LINE_IN', 'wire unsigned', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, ''],
    ['OUT_MUX_SEL_DIMENSION', 'wire unsigned', int(math.log2((FILTERS+1))), ''],
    ['OUT_MUX_SEL_FILTER_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_MUX_SEL_FILTER_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_ADD_COUNTER', 'wire unsigned', 4, ''],
    ['EXT_OUT_REG_INPUT_LINE', 'wire unsigned', OUT_1ST_INTERP_WIDTH*INPUT_LINE_SIZE, '\t\t// Sign extension for OUT_REG_INPUT_LINE (72 -> 99 bits)', ''],

    ['OUT_INTERP_0_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_0_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_1_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_1_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_2_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_2_14', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],

    ['OUT_INTERP_3_0', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_1', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_2', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_3', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_4', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_5', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_6', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_7', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_8', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_9', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_10', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_11', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_12', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_13', 'wire signed', OUT_2ND_INTERP_WIDTH, ''],
    ['OUT_INTERP_3_14', 'wire signed', OUT_2ND_INTERP_WIDTH, '']
]


# Components informations 
# [component name, component type, [inputs names], [outputs], description] 
# Template: ['', '', [], , [], , ''],
INT_COMPONENTS = [
    ['REG_INPUT_LINE', 'reg_input_line', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INPUT_LINE', 'INTEGER_SAMPLES', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_REG_INPUT_LINE', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Stores the external input samples'],

    ['REG_COUNTER', 'reg_counter', ['CLK', 'RST_ASYNC_REG_COUNTER', 'WRITE_REG_COUNTER', 'OUT_ADD_COUNTER', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_REG_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Controls the counter increment that controls the loops for FSM'],

    ['TRANSPOSED_BUFFER', 'transposed_buffer', 
     ['CLK', 'RST_ASYNC_INTERP', 'WRITE_BUFFER', 
      'OUT_MUX_SEL_BUF_IN_0', 'OUT_MUX_SEL_BUF_IN_1', 'OUT_MUX_SEL_BUF_IN_2', 'OUT_MUX_SEL_BUF_IN_3', '', '', '', '', '', '', '', '', ''], 
      ['OUT_BUFFER_0', 'OUT_BUFFER_1', 'OUT_BUFFER_2', 'OUT_BUFFER_3', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['REG_INT_OUT_0', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_0', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 0'],

    ['REG_INT_OUT_1', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_1', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 1'],

    ['REG_INT_OUT_2', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_2', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 2'],

    ['REG_INT_OUT_3', 'reg_int_out', ['CLK', 'RST_ASYNC_INTERP', 'WRITE_REG_INT_OUT', 'OUT_MUX_SEL_BUF_IN_3', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['INTERP_OUT_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t\t// Output from interpolator 3'],

    ['ADD_COUNTER', 'add_counter', ["4'b1", 'OUT_REG_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_ADD_COUNTER', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_0', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_0', 'OUT_REG_INPUT_LINE[55:48]', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_1', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_1', 'OUT_REG_INPUT_LINE[47:40]', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_2', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_2', 'OUT_REG_INPUT_LINE[39:32]', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUF_IN_3', 'mux_sel_buffer_in', ['OUT_MUX_SEL_FILTER_3', 'OUT_REG_INPUT_LINE[31:24]', 'SEL_BUFFER_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_BUF_IN_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_BUFFER_COLUMN', 'mux_sel_buffer_column', ['OUT_BUFFER_0', 'OUT_BUFFER_1', 'OUT_BUFFER_2', 'OUT_BUFFER_3', 'OUT_REG_COUNTER[2:0]', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_MUX_SEL_BUFFER_COLUMN', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_INTERP_LINE_IN', 'mux_sel_interp_line_in', ['OUT_MUX_SEL_BUFFER_COLUMN', 'EXT_OUT_REG_INPUT_LINE', 'SEL_INTERP_IN', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_MUX_SEL_INTERP_LINE_IN', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_DIMENSION', 'mux_sel_dimension', ['FRAC_MV_Y', 'FRAC_MV_X', 'SEL_DIMENSION', '', '', '', '', '', '', '', '', '', '', '', '', ''], 
     ['OUT_MUX_SEL_DIMENSION', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], '\t\t// The output selects the filter'],

    ['MUX_SEL_FILTER_0', 'mux_sel_filter', ['OUT_INTERP_0_0', 'OUT_INTERP_0_1', 'OUT_INTERP_0_2', 'OUT_INTERP_0_3', 'OUT_INTERP_0_4', 
                                            'OUT_INTERP_0_5', 'OUT_INTERP_0_6', 'OUT_INTERP_0_7', 'OUT_INTERP_0_8', 'OUT_INTERP_0_9', 
                                            'OUT_INTERP_0_10', 'OUT_INTERP_0_11', 'OUT_INTERP_0_12', 'OUT_INTERP_0_13', 'OUT_INTERP_0_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_0', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_1', 'mux_sel_filter', ['OUT_INTERP_1_0', 'OUT_INTERP_1_1', 'OUT_INTERP_1_2', 'OUT_INTERP_1_3', 'OUT_INTERP_1_4', 
                                            'OUT_INTERP_1_5', 'OUT_INTERP_1_6', 'OUT_INTERP_1_7', 'OUT_INTERP_1_8', 'OUT_INTERP_1_9', 
                                            'OUT_INTERP_1_10', 'OUT_INTERP_1_11', 'OUT_INTERP_1_12', 'OUT_INTERP_1_13', 'OUT_INTERP_1_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_1', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_2', 'mux_sel_filter', ['OUT_INTERP_2_0', 'OUT_INTERP_2_1', 'OUT_INTERP_2_2', 'OUT_INTERP_2_3', 'OUT_INTERP_2_4', 
                                            'OUT_INTERP_2_5', 'OUT_INTERP_2_6', 'OUT_INTERP_2_7', 'OUT_INTERP_2_8', 'OUT_INTERP_2_9', 
                                            'OUT_INTERP_2_10', 'OUT_INTERP_2_11', 'OUT_INTERP_2_12', 'OUT_INTERP_2_13', 'OUT_INTERP_2_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_2', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['MUX_SEL_FILTER_3', 'mux_sel_filter', ['OUT_INTERP_3_0', 'OUT_INTERP_3_1', 'OUT_INTERP_3_2', 'OUT_INTERP_3_3', 'OUT_INTERP_3_4', 
                                            'OUT_INTERP_3_5', 'OUT_INTERP_3_6', 'OUT_INTERP_3_7', 'OUT_INTERP_3_8', 'OUT_INTERP_3_9', 
                                            'OUT_INTERP_3_10', 'OUT_INTERP_3_11', 'OUT_INTERP_3_12', 'OUT_INTERP_3_13', 'OUT_INTERP_3_14',
                                            'OUT_MUX_SEL_DIMENSION'],
     ['OUT_MUX_SEL_FILTER_3', '', '', '', '', '', '', '', '', '', '', '', '', '', ''], ''],

    ['INTERP_0', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[98:33]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
      ['OUT_INTERP_0_0', 'OUT_INTERP_0_1', 'OUT_INTERP_0_2', 'OUT_INTERP_0_3', 'OUT_INTERP_0_4', 
       'OUT_INTERP_0_5', 'OUT_INTERP_0_6', 'OUT_INTERP_0_7', 'OUT_INTERP_0_8', 'OUT_INTERP_0_9', 
       'OUT_INTERP_0_10', 'OUT_INTERP_0_11', 'OUT_INTERP_0_12', 'OUT_INTERP_0_13', 'OUT_INTERP_0_14'], ''],

    ['INTERP_1', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[87:22]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_1_0', 'OUT_INTERP_1_1', 'OUT_INTERP_1_2', 'OUT_INTERP_1_3', 'OUT_INTERP_1_4', 
      'OUT_INTERP_1_5', 'OUT_INTERP_1_6', 'OUT_INTERP_1_7', 'OUT_INTERP_1_8', 'OUT_INTERP_1_9', 
      'OUT_INTERP_1_10', 'OUT_INTERP_1_11', 'OUT_INTERP_1_12', 'OUT_INTERP_1_13', 'OUT_INTERP_1_14'], ''],

    ['INTERP_2', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[76:11]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_2_0', 'OUT_INTERP_2_1', 'OUT_INTERP_2_2', 'OUT_INTERP_2_3', 'OUT_INTERP_2_4', 
      'OUT_INTERP_2_5', 'OUT_INTERP_2_6', 'OUT_INTERP_2_7', 'OUT_INTERP_2_8', 'OUT_INTERP_2_9', 
      'OUT_INTERP_2_10', 'OUT_INTERP_2_11', 'OUT_INTERP_2_12', 'OUT_INTERP_2_13', 'OUT_INTERP_2_14'], ''],

    ['INTERP_3', 'all_filters_11', ['OUT_MUX_SEL_INTERP_LINE_IN[65:0]', '', '', '', '', '', '', '', '', '', '', '', '', '', ''],
     ['OUT_INTERP_3_0', 'OUT_INTERP_3_1', 'OUT_INTERP_3_2', 'OUT_INTERP_3_3', 'OUT_INTERP_3_4', 
      'OUT_INTERP_3_5', 'OUT_INTERP_3_6', 'OUT_INTERP_3_7', 'OUT_INTERP_3_8', 'OUT_INTERP_3_9', 
      'OUT_INTERP_3_10', 'OUT_INTERP_3_11', 'OUT_INTERP_3_12', 'OUT_INTERP_3_13', 'OUT_INTERP_3_14'], '']
]
 


In [7]:
####################################################
# MV GENERATOR DATAPATH CALCULATIONS
####################################################

################## Inputs declaration ##################
IO_DECLARATION = []


################## Signals declaration ##################
SIG_DECLARATION = []


################## Components declarations ##################
COMPONENTS_DECLARATION = []


################## Combinational logic ##################
COMBINATIONAL_LOGIC = []
COMBINATIONAL_LOGIC.append('  EXT_OUT_REG_INPUT_LINE = {'+str((OUT_1ST_INTERP_WIDTH - PIXEL_WIDTH) * INPUT_LINE_SIZE)+'{OUT_REG_INPUT_LINE['+str(PIXEL_WIDTH*INPUT_LINE_SIZE-1)+']}, OUT_REG_INPUT_LINE};')

COMBINATIONAL_LOGIC_CONCAT = '\n'.join(COMBINATIONAL_LOGIC)


# print(IO_DECLARATION_CONCAT)
# print(SIG_DECLARATION_CONCAT)
# print(COMPONENTS_DECLARATION_CONCAT)
# print(COMBINATIONAL_LOGIC_CONCAT)

  EXT_OUT_REG_INPUT_LINE = {27{OUT_REG_INPUT_LINE[71]}, OUT_REG_INPUT_LINE};


### Datapath text

In [4]:
#########################################
# Text writing: Interpolation datapath
#########################################
INT_DATAPATH_VERILOG = '''/*------------------------------------------------------------------------------
 * File: '''+INT_FILE_NAME+'''.v
 * Date generated: 13/03/2023
 * Date modified: '''+DATE_MODIF+'''
 * Author: Bruna Suemi Nagai
 * Description: '''+INT_DESCRIPTION+'''
 *------------------------------------------------------------------------------ */

module '''+INT_FILE_NAME+''' (
'''+IO_DECLARATION_CONCAT+'''
);


// ------------------------------------------
// Signals declarations
// ------------------------------------------
'''+SIG_DECLARATION_CONCAT+'''

// ------------------------------------------
// Modules instantiation
// ------------------------------------------
'''+COMPONENTS_DECLARATION_CONCAT+'''
    

// ------------------------------------------
// Combinational logic
// ------------------------------------------
'''+COMBINATIONAL_LOGIC_CONCAT+'''


endmodule // '''+MV_GEN_FILE_NAME+'''    
'''

print(INT_DATAPATH_VERILOG)
# with open('../verilog/datapaths/'+INT_FILE_NAME+'.v', 'w') as int_datapath_file:
#     int_datapath_file.write(INT_DATAPATH_VERILOG)